# Homework 2

## FINM 37400 - 2025

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

***

# 1 HBS Case: Fixed-Income Arbitrage in a Financial Crisis (A): US Treasuries in November 2008

## Data
* Use the data file `treasury_ts_2015-08-15.xlsx`.
* Examine the treasure issues with `kytreasno` of `204046` and `204047`. These are the bond and note (respectively) which mature on 2015-08-15.
* Look at the data on 2008-11-04.

## 1.1 The situation

Make a chart comparing the issues in the following features, (as of Nov 4, 2008.)
* coupon rate
* bid
* ask
* accrued interest
* dirty price
* duration (quoted in years, not days, assuming 365.25 days per year.)
* modified duration
* YTM

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [35]:
df = pd.read_excel("../data/treasury_ts_2015-08-15.xlsx",sheet_name="database")
df_filtered = df[((df['kytreasno']==204046) | (df['kytreasno']==204047)) & (df['caldt'] == '2008-11-04')]
df_filtered

,kytreasno,kycrspid,caldt,tdbid,tdask,tdnomprc,tdnomprc_flg,tdsourcr,tdaccint,tdretnua,tdyld,tdduratn,tdpubout,tdtotout,tdpdint,tdidxratio,tdidxratio_flg
4178,204047,2.015082e+07,2008-11-04,105.953125,105.984375,105.96875,M,X,0.935462,0.011642,0.000089,2168.016569,20998.0,32470.0,0.0,NaN,NaN
5834,204046,2.015082e+07,2008-11-04,141.859375,141.890625,141.87500,M,X,2.338655,0.009720,0.000098,1910.307862,2852.0,4024.0,0.0,NaN,NaN


In [36]:
pd.options.mode.chained_assignment = None

df_filtered['Duration (Years)'] = df_filtered['tdduratn'] / 365.25
df_filtered['Modified Duration'] = df_filtered['Duration (Years)'] / (1 + df_filtered['tdyld'])

features = df_filtered[['kytreasno', 'tdbid', 'tdask', 'tdaccint', 'tdnomprc', 
                        'Duration (Years)', 'Modified Duration', 'tdyld']]

features.rename(columns={
    'kytreasno': 'Treasury Issue',
    'tdbid': 'Bid Price',
    'tdask': 'Ask Price',
    'tdaccint': 'Accrued Interest',
    'tdnomprc': 'Dirty Price',
    'tdyld': 'Yield to Maturity (YTM)',
}, inplace=True)

features

,Treasury Issue,Bid Price,Ask Price,Accrued Interest,Dirty Price,Duration (Years),Modified Duration,Yield to Maturity (YTM)
4178,204047,105.953125,105.984375,0.935462,105.96875,5.935706,5.935180,0.000089
5834,204046,141.859375,141.890625,2.338655,141.87500,5.230138,5.229626,0.000098


## 1.2 Hedge Ratio

Suppose you are building a trade to go long $n_i$ bonds (`204046`) and short $n_j$ notes (`204047`).

We can find an equation for $n_j$ in terms of $n_i$ such that the total holdings will have duration equal to zero. (Having zero duration also means have zero dollar duration, if helpful.)

Notation:
* $n_i$: number of bonds purchased (or sold)
* $D_i$: duration of bond $i$
* $D_{\$,i}$: dollar duration of bond $i$, equal to $p_iD_i$

If we want the total duration of our holdings to be zero, then we need to size the trade such that $n_i$ and $n_j$ satisfy,

$$0 = n_iD_{\$,i} + n_jD_{\$,j}$$

$$n_j = -n_i\frac{D_{\$,i}}{D_{\$,j}}$$

Suppose you will use \\$1mm of capital, leveraged 50x to buy \\$50mm of the bonds (`204046`).

Use the ratio above to short a number of notes (`204047`) to keep zero duration.

Report the number of bonds and notes of your position, along with the total dollars in the short position.

In [37]:
bond_204046 = features[features['Treasury Issue'] == 204046].iloc[0]
bond_204047 = features[features['Treasury Issue'] == 204047].iloc[0]

p_i = bond_204046['Dirty Price']
D_i = bond_204046['Duration (Years)']

p_j = bond_204047['Dirty Price']
D_j = bond_204047['Duration (Years)']

capital_used = 50_000_000  

D_dollar_i = p_i * D_i
D_dollar_j = p_j * D_j

n_i = capital_used / p_i
n_j = -n_i * (D_dollar_i / D_dollar_j)

short_position_dollars = abs(n_j * p_j)

print(f"Number of bonds purchased (204046): {n_i:,.2f}")
print(f"Number of notes shorted (204047): {n_j:,.2f}")
print(f"Total dollars in short position: ${short_position_dollars:,.2f}")

Number of bonds purchased (204046): 352,422.91
Number of notes shorted (204047): -415,750.67
Total dollars in short position: $44,056,578.93


## 1.3 Profit Opportunity

Using the concept of **modified duration**, how much profit or loss (PnL) would you expect to make for every basis point of convergence in the spread? Specifically, assume the convergence is symmetric: the bond's (`204046`) ytm goes down 0.5bp and the note (`204047`) ytm goes up 0.5bp.

Describe the PnL you would expect to achieve on your position should this happen. Specify the PnL of the long position, the short position, and the net total.

Suppose the spread in YTM between the two securities disappears, due to a symmetric move of roughly ~17bps in each security's YTM. What is the PnL? (This is just a linearly scaling of your prior answer for a 1bp convergence.) 


In [82]:
md_i = bond_204046['Modified Duration']
md_j = bond_204047['Modified Duration']

capital_used = 50_000_000
n_i = capital_used / p_i
n_j = -n_i * (p_i * md_i) / (p_j * md_j)

bp_move = 0.5 / 10000

#1bp move
pnl_long_1bp = md_i * p_i * n_i * bp_move
pnl_short_1bp = -md_j * p_j * n_j * bp_move
pnl_total_1bp = pnl_long_1bp + pnl_short_1bp

#17bp move
pnl_long_17bp = pnl_long_1bp * 17
pnl_short_17bp = pnl_short_1bp * 17
pnl_total_17bp = pnl_total_1bp * 17

print(f"PnL for a 1bp convergence:")
print(f"  - Long Bond (204046) PnL: ${pnl_long_1bp:,.2f}")
print(f"  - Short Note (204047) PnL: ${pnl_short_1bp:,.2f}")
print(f"  - Total PnL: ${pnl_total_1bp:,.2f}")

print("\nPnL for a 17bp convergence:")
print(f"  - Long Bond (204046) PnL: ${pnl_long_17bp:,.2f}")
print(f"  - Short Note (204047) PnL: ${pnl_short_17bp:,.2f}")
print(f"  - Total PnL: ${pnl_total_17bp:,.2f}")

PnL for a 1bp convergence:
  - Long Bond (204046) PnL: $13,074.06
  - Short Note (204047) PnL: $13,074.06
  - Total PnL: $26,148.13

PnL for a 17bp convergence:
  - Long Bond (204046) PnL: $222,259.10
  - Short Note (204047) PnL: $222,259.10
  - Total PnL: $444,518.21


## 1.4 Result in 2008

Calculate the profit (or loss) on the position on the following two dates:
* 2008-11-25
* 2008-12-16

To calculate the pnl on each date, simply use the prices of the securities on those dates along with your position sizes, ($n_i, n_j$). No coupon is being paid in November or December, so all you need is the "dirty" price on these two dates.

Does the pnl make sense (approximately) given your results in 1.3 with regard to the sensitivity of pnl to moves in the YTM spread?

In [60]:
df_filtered_11_25 = df[((df['kytreasno']==204046) | (df['kytreasno']==204047)) & (df['caldt'] == '2008-11-25')]
df_filtered_11_25

,kytreasno,kycrspid,caldt,tdbid,tdask,tdnomprc,tdnomprc_flg,tdsourcr,tdaccint,tdretnua,tdyld,tdduratn,tdpubout,tdtotout,tdpdint,tdidxratio,tdidxratio_flg
4192,204047,2.015082e+07,2008-11-25,110.796875,110.828125,110.8125,M,X,1.177989,0.011821,0.000068,2155.590747,20998.0,32470.0,0.0,NaN,NaN
5848,204046,2.015082e+07,2008-11-25,145.859375,145.890625,145.8750,M,X,2.944973,0.006642,0.000082,1898.970318,2852.0,4024.0,0.0,NaN,NaN


In [61]:
df_filtered_12_26 = df[((df['kytreasno']==204046) | (df['kytreasno']==204047)) & (df['caldt'] == '2008-12-26')]
df_filtered_12_26

,kytreasno,kycrspid,caldt,tdbid,tdask,tdnomprc,tdnomprc_flg,tdsourcr,tdaccint,tdretnua,tdyld,tdduratn,tdpubout,tdtotout,tdpdint,tdidxratio,tdidxratio_flg
4213,204047,2.015082e+07,2008-12-26,116.81250,116.84375,116.828125,M,X,1.536005,0.001915,0.000043,2134.622761,20998.0,32470.0,0.0,NaN,NaN
5869,204046,2.015082e+07,2008-12-26,151.78125,151.81250,151.796875,M,X,3.840014,0.002586,0.000060,1881.774790,2852.0,4024.0,0.0,NaN,NaN


In [63]:
pnl_i = n_i * (df_filtered_12_26[df_filtered_12_26['kytreasno']==204046]['tdnomprc'].values - df_filtered_11_25[df_filtered_11_25['kytreasno']==204046]['tdnomprc'].values)
pnl_j = n_j * (df_filtered_12_26[df_filtered_12_26['kytreasno']==204047]['tdnomprc'].values - df_filtered_11_25[df_filtered_11_25['kytreasno']==204047]['tdnomprc'].values)
pnl_total = pnl_i[0] + pnl_j[0]

print(f"PnL for Bond 204046 (Long) from 2008-11-25 to 2008-12-16: ${pnl_i[0]:,.2f}")
print(f"PnL for Bond 204047 (Short) from 2008-11-25 to 2008-12-16: ${pnl_j[0]:,.2f}")
print(f"Total PnL for the Position: ${pnl_total:,.2f}")

PnL for Bond 204046 (Long) from 2008-11-25 to 2008-12-16: $2,087,004.41
PnL for Bond 204047 (Short) from 2008-11-25 to 2008-12-16: $-2,500,976.92
Total PnL for the Position: $-413,972.51


Yes, this generally makes sense. Since we hedged the positions earlier, price variations should somewhat cancel each other out. We can see that even though they moved over $2M PNL each, the net PNL was only $400k, which shows our hedging is somewhat effective.

## 1.5 Examining the Trade through June 2009

Calculate the pnl of the trade for the following dates:
* 2009-01-27
* 2009-03-24
* 2009-06-16

Did the trade do well or poorly in the first six months of 2009?

Calculate the YTM spreads on these dates. Does the YTM spread correspond to pnl roughly as we would expect based on the calculation in 1.3?

In [65]:
df_filtered_01_27 = df[((df['kytreasno']==204046) | (df['kytreasno']==204047)) & (df['caldt'] == '2009-01-27')]
df_filtered_03_24 = df[((df['kytreasno']==204046) | (df['kytreasno']==204047)) & (df['caldt'] == '2009-03-24')]
df_filtered_06_16 = df[((df['kytreasno']==204046) | (df['kytreasno']==204047)) & (df['caldt'] == '2009-06-16')]

In [81]:
# First, gather your start/end DataFrames, date labels, etc. into one list of tuples.
# For example:
trade_data = [
    (df_filtered_12_26, df_filtered_01_27, "2008-12-26", "2009-01-27"),
    (df_filtered_01_27, df_filtered_03_24, "2009-01-27", "2009-03-24"),
    (df_filtered_03_24, df_filtered_06_16, "2009-03-24", "2009-06-16"),
    # Add more pairs as needed
]

# For clarity, define the 'kytreasno' values for the two bonds you mention:
BOND_LONG = 204046
BOND_SHORT = 204047

# Now loop over each pair of DataFrames
for df_start, df_end, start_label, end_label in trade_data:
    
    # Compute PnL for the 'long' bond (204046)
    pnl_i = n_i * (
        df_end[df_end["kytreasno"] == BOND_LONG]["tdnomprc"].values
        - df_start[df_start["kytreasno"] == BOND_LONG]["tdnomprc"].values
    )

    # Compute PnL for the 'short' bond (204047)
    pnl_j = n_j * (
        df_end[df_end["kytreasno"] == BOND_SHORT]["tdnomprc"].values
        - df_start[df_start["kytreasno"] == BOND_SHORT]["tdnomprc"].values
    )

    # Total
    pnl_total = pnl_i + pnl_j

    # Yield spread
    ytm_spread = (
        df_end[df_end["kytreasno"] == BOND_SHORT]["tdyld"].values
        - df_end[df_end["kytreasno"] == BOND_LONG]["tdyld"].values
    )

    # Print out results
    print(f"PnL for Bond {BOND_LONG} (Long) from {start_label} to {end_label}: {pnl_i[0]:,.2f}")
    print(f"PnL for Bond {BOND_SHORT} (Short) from {start_label} to {end_label}: {pnl_j[0]:,.2f}")
    print(f"Total PnL for the Position: {pnl_total[0]:,.2f}")
    print(f"YTM Spread: {ytm_spread[0]:.6f}%\n")


PnL for Bond 204046 (Long) from 2008-12-26 to 2009-01-27: -468,061.67
PnL for Bond 204047 (Short) from 2008-12-26 to 2009-01-27: 1,084,839.34
Total PnL for the Position: 616,777.66
YTM Spread: -0.000010%

PnL for Bond 204046 (Long) from 2009-01-27 to 2009-03-24: -206,497.80
PnL for Bond 204047 (Short) from 2009-01-27 to 2009-03-24: 308,562.09
Total PnL for the Position: 102,064.29
YTM Spread: -0.000006%

PnL for Bond 204046 (Long) from 2009-03-24 to 2009-06-16: -2,513,766.52
PnL for Bond 204047 (Short) from 2009-03-24 to 2009-06-16: 2,829,027.14
Total PnL for the Position: 315,260.62
YTM Spread: 0.000002%



not sure about the essay portion here.

***

# 2 Hedging Duration

Use data from `../data/treasury_ts_duration_2024-10-31.xlsx`.

The file contains time-series information on two treasuries. Observe the info of the securities with the following code:


In [40]:
import pandas as pd

In [41]:
QUOTE_DATE = '2024-10-31'
filepath = f'../data/treasury_ts_duration_{QUOTE_DATE}.xlsx'

data = pd.read_excel(filepath,sheet_name='database')
data_info =  data.drop_duplicates(subset='KYTREASNO', keep='first').set_index('KYTREASNO')
data_info[['type','issue date','maturity date','cpn rate']]

,type,issue date,maturity date,cpn rate
KYTREASNO,,,,
207391,note,2019-08-15,2029-08-15,1.625
207392,bond,2019-08-15,2049-08-15,2.250


You will largely focus on the sheets which give the timeseries of prices and durations for each of the two securities, as shown in the following code.

In [42]:
SHEET_PRICE = 'price'
SHEET_DURATION = 'duration'
INDEX_NAME = 'quote date'

price = pd.read_excel(filepath,sheet_name=SHEET_PRICE).set_index(INDEX_NAME)
duration = pd.read_excel(filepath,sheet_name=SHEET_DURATION).set_index(INDEX_NAME)

display(price)
display(duration)

,207391,207392
quote date,,
2019-08-09,98.882812,99.789062
2019-08-12,99.796875,102.554688
2019-08-13,99.281250,101.867188
2019-08-14,100.406250,105.179688
2019-08-15,100.882812,106.234375
...,...,...
2024-11-22,88.740234,63.722656
2024-11-25,89.287109,65.378906
2024-11-26,89.214844,65.175781


,207391,207392
quote date,,
2019-08-09,9.289497,22.000102
2019-08-12,9.285468,22.118496
2019-08-13,9.280314,22.084308
2019-08-14,9.282750,22.228549
2019-08-15,9.282163,22.270910
...,...,...
2024-11-22,4.539445,17.205511
2024-11-25,4.531983,17.312267
2024-11-26,4.529132,17.295472


### 2.1.

Suppose you have a portfolio of `10,000` USD long in security `207391` on the first day of the sample.

If you want to manage interest rate exposure using duration, how large of a short position should you hold in `207392`?

### 2.2.

Step through the time-series, doing the following:

* Starting at the end of the first day, set the hedged position according to the relative given durations.
* Use the second day's price data to evaluate the net profit or loss of the hedged position.
* Reset the the hedged position using the end-of-second-day durations. Again fix the long position of security `207391` to be `10,000`.
* Repeat throughout the timeseries.

Calculate the daily profit and loss (PnL) for the
* dynamically hedged position constructed above.
* long-only position, (still at `10,000` throughout.)

(You might check to verify that the net duration is zero at all dates.)

Report...
* the cumulative PnL of both strategies via a plot.
* the (daily) mean, standard deviation, min, and max of the PnL in a table.

### 2.3.

Give two reasons that the daily PnL is not always zero for the hedged position given that we have perfectly hedged the duration.

### 2.4.
The PnL above doesn't account for the coupons.

Calculate a dataframe indexed by dates with columns for the two treasuries with values of coupon payments. 
* Recall that the stated coupon rate is semiannual, so at any give coupon date, it pays half the stated rate.
* Figure out the coupon dates by using the `data` tab and looking for dates where `acc int` goes down. Recall that accrued interest measures the portion of the coupon period that has passed. So when this resets, it is because the coupon has been paid.

Report the first 5 dates that a coupon is paid (by either bond).

### 2.5.
Account for the coupons in the PnL calculations of `2.2`. Report the updated PnL in a plot and a table, similar to the reporting in `2.2`.

***